In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())


In [4]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [7]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__() # calls the init method of the parent class
        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=num_classes)
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        w1 = self.fc1(x)
        w1_relu = self.relu(w1)
        out = self.fc2(w1_relu)
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes)

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
total_step = len(train_loader)
print(total_step)

600


In [31]:
for epoch in range(num_epochs):
    for batch_id, (images, labels) in enumerate(train_loader):
        # move tensors to device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) #unhappiness with the outputs
        
        # backward pass
        criterion.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_id+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, batch_id+1, total_step, loss.item()))
        

Epoch [1/5], Step [100/600], Loss: 0.3822
Epoch [1/5], Step [200/600], Loss: 0.1514
Epoch [1/5], Step [300/600], Loss: 0.3463
Epoch [1/5], Step [400/600], Loss: 0.5178
Epoch [1/5], Step [500/600], Loss: 0.5273
Epoch [1/5], Step [600/600], Loss: 0.1590
Epoch [2/5], Step [100/600], Loss: 0.3451
Epoch [2/5], Step [200/600], Loss: 0.6112
Epoch [2/5], Step [300/600], Loss: 0.9073
Epoch [2/5], Step [400/600], Loss: 0.1736
Epoch [2/5], Step [500/600], Loss: 0.5892
Epoch [2/5], Step [600/600], Loss: 0.8209
Epoch [3/5], Step [100/600], Loss: 0.1010
Epoch [3/5], Step [200/600], Loss: 0.5753
Epoch [3/5], Step [300/600], Loss: 0.2727
Epoch [3/5], Step [400/600], Loss: 0.3438
Epoch [3/5], Step [500/600], Loss: 0.3687
Epoch [3/5], Step [600/600], Loss: 0.5944
Epoch [4/5], Step [100/600], Loss: 0.4179
Epoch [4/5], Step [200/600], Loss: 0.2793
Epoch [4/5], Step [300/600], Loss: 0.4965
Epoch [4/5], Step [400/600], Loss: 0.5676
Epoch [4/5], Step [500/600], Loss: 0.4396
Epoch [4/5], Step [600/600], Loss:

In [47]:
### Test
correct = 0
total = 0
with torch.no_grad():
    for batch_id, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        total += len(labels)
        
        # energies for the 10 classes
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of the network on the 10000 test images: {correct/total*100}')
        
    

Accuracy of the network on the 10000 test images: 88.34
